# 🤖 การเทรน AI ตรวจจับวัตถุด้วย YOLOv8 - โรงเรียนลำปลายมาศ

ยินดีต้อนรับนักเรียนเข้าสู่บทเรียนการสร้าง AI Computer Vision!
ในสมุดโน้ต (Notebook) นี้ เราจะเรียนรู้วิธีการเทรนโมเดล YOLOv8 เพื่อตรวจจับวัตถุ โดยใช้ข้อมูลจาก Roboflow

---

## 1. ตรวจสอบการ์ดจอ (GPU)
เพื่อให้การเทรนรวดเร็ว เราต้องใช้ GPU ของ Google Colab
ถ้าคำสั่งด้านล่างไม่แสดงข้อมูล GPU ให้ไปที่เมนู **Runtime > Change runtime type > เลือก T4 GPU**

In [ ]:
!nvidia-smi

## 2. ติดตั้งเครื่องมือ (Install YOLOv8)
เราจะติดตั้ง Library `ultralytics` ซึ่งเป็นผู้พัฒนา YOLOv8

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from IPython.display import display, Image
import os

## 3. นำเข้าข้อมูล (Dataset) จาก Roboflow
ให้นักเรียนไปที่โปรเจคใน Roboflow ของตัวเอง 
1. กด **Export Dataset** 
2. เลือก format เป็น **YOLOv8** 
3. เลือก **Show Download Code** 
4. คัดลอกโค้ด Python มาวางแทนที่ในช่องด้านล่างนี้

In [ ]:
# รีเซ็ต path ไปที่ /content เสมอ เพื่อป้องกัน error เวลา run ซ้ำ
%cd /content
!mkdir -p datasets
%cd datasets
%pip install roboflow

from roboflow import Roboflow

# --- ⬇️ วางโค้ดจาก Roboflow ด้านล่างนี้ (ลบโค้ดตัวอย่างออกก่อน) ⬇️ ---

# ตัวอย่าง (อย่าลืมเปลี่ยนเป็นโค้ดของตัวเอง):
# rf = Roboflow(api_key="YOUR_API_KEY_HERE")
# project = rf.workspace("workspace-name").project("project-name")
# version = project.version(1)
# dataset = version.download("yolov8")

# ----------------------------------------------------------------

## 4. เริ่มการเทรน (Training)
กำหนดจำนวนรอบ (Epochs) ในการเรียนรู้ ยิ่งเยอะยิ่งเรียนรู้นานและอาจจะเก่งขึ้น (แต่ถ้าเยอะไปอาจจะจำข้อสอบ หรือ Overfitting)

In [ ]:
#@title ⚙️ กำหนดค่าการเทรน
epochs_value = 50 #@param {type:"integer"}
img_size = 640 #@param {type:"integer"}
batch_size = 16 #@param {type:"integer"}

# กลับไปที่โฟลเดอร์หลัก
%cd /content

import os
from glob import glob

# ตรวจสอบหาไฟล์ data.yaml อัตโนมัติ แก้ปัญหา path ผิด
data_yaml_path = None

if 'dataset' in locals() and hasattr(dataset, 'location'):
    # กรณีมีตัวแปร dataset จาก Roboflow
    potential_path = f"{dataset.location}/data.yaml"
    if os.path.exists(potential_path):
        data_yaml_path = potential_path
    else:
        print(f"⚠️ ไม่พบไฟล์ที่ {potential_path} กำลังค้นหาใหม่...")

if data_yaml_path is None:
    # ค้นหาไฟล์ data.yaml ใน folder datasets ทั้งหมด
    found_files = glob("/content/datasets/**/data.yaml", recursive=True)
    if found_files:
        data_yaml_path = found_files[0]
        print(f"✅ ค้นพบไฟล์ data.yaml ที่: {data_yaml_path}")
    else:
        print("❌ ไม่พบไฟล์ data.yaml! กรุณาตรวจสอบว่าได้วางโค้ด Roboflow และรัน Cell ที่ 3 หรือยัง")

if data_yaml_path:
    print(f"🚀 กำลังเริ่มเทรนโมเดลจำนวน {epochs_value} รอบ...")
    # รันคำสั่งเทรนโดยใช้ path ที่หาเจอ
    !yolo task=detect mode=train model=yolov8n.pt data="{data_yaml_path}" epochs={epochs_value} imgsz={img_size} batch={batch_size} plots=True
else:
    print("⛔ หยุดการทำงาน: ไม่สามารถเริ่มเทรนได้เนื่องจากไม่พบไฟล์ข้อมูล")

## 5. ดูผลลัพธ์การเทรน (Results)
ดูกราฟค่าความสูญเสีย (Loss) และความแม่นยำ (mAP)

In [ ]:
from IPython.display import Image, display
Image(filename=f'runs/detect/train/results.png', width=800)

ดูตาราง Confusion Matrix เพื่อดูว่าทายผิด/ถูก อย่างไรบ้าง

In [ ]:
from IPython.display import Image, display
Image(filename=f'runs/detect/train/confusion_matrix.png', width=800)

## 6. ทดสอบโมเดลกับภาพตัวอย่าง (Validation Batch)
ดูว่าโมเดลทำนายภาพในชุดข้อมูลตรวจสอบ (Validation Set) อย่างไร

In [ ]:
from IPython.display import Image, display
Image(filename=f'runs/detect/train/val_batch0_pred.jpg', width=800)

## 7. ดาวน์โหลดโมเดล (Download best.pt)
ดาวน์โหลดไฟล์ `best.pt` เพื่อนำไปใช้งานในโปรแกรมอื่น หรือบนเว็บไซต์

In [ ]:
from google.colab import files
files.download('runs/detect/train/weights/best.pt')